# 第2章：人在环路 - 中断审查及持久化

## 本章目标

本章将实现人机协同机制，让Agent能够在需要时暂停执行，等待人工决策。你将学习：

- **interrupt()**: 中断执行，等待人工输入
- **Command**: 动态路由和状态更新
- **MemorySaver**: 状态持久化
- **中断恢复**: 如何在中断后继续执行
- **两种人工协同模式**: 硬编码 vs LLM驱动

## 应用场景

- **敏感操作确认**: 删除文件、发送邮件前需要确认
- **需求澄清**: Agent不确定用户意图时询问
- **人工审核**: 在执行重要决策前获得批准
- **信息补充**: 向用户请求额外信息


## 💡 这是 Claude Code 的什么功能？

这是 Claude Code 的**权限确认机制**。当 Claude 要执行敏感操作时，会暂停并询问你：

```
Claude wants to:
✓ Read config.json
✓ Edit database.py (line 45-67)
✓ Run: rm -rf node_modules

[Approve] [Reject] [Edit]
```

**为什么需要**：
- 防止 AI 误删文件、执行危险命令
- 让你保持对代码的控制权
- 可以修改 AI 的建议再执行

**技术要点**：
- 使用 `interrupt()` 暂停 graph 执行
- 使用 `Command` 动态路由到不同节点
- 使用 checkpointer 保存中断时的状态
- 提供 `ask_human` 工具，让 **LLM 自己判断**何时需要询问用户。

本章实现的 `interrupt()` 和 `Command`，就是这个功能的技术基础。

## 1. 环境设置

In [ ]:
import os
from typing import Literal
from IPython.display import Image, display

# LangGraph核心
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command, interrupt
from langgraph.prebuilt import ToolNode, tools_condition

# LangChain
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage

# LLM
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatTongyi

In [ ]:
# 配置
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "chapter-02-human-in-the-loop"

# 初始化LLM
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# 选项2: 通义千问 (取消注释使用)
llm = ChatTongyi(model="qwen3-max", temperature=0)
print(f"✅ LLM初始化成功: {llm.model_name}")

## 2. 基础示例：使用interrupt()暂停执行

### 核心概念

`interrupt()` 函数可以暂停图的执行，并将数据返回给调用者。要恢复执行，需要使用 `Command(resume=...)`。

**重要**: 必须配置checkpointer才能使用interrupt！

In [ ]:
from typing import TypedDict

# 定义简单的状态
class SimpleState(TypedDict):
    text: str


def human_review_node(state: SimpleState):
    """人工审查节点
    
    使用interrupt()暂停执行，等待人工输入
    """
    print(f"📋 当前文本: {state['text']}")
    
    # 中断执行，返回需要审查的内容
    # interrupt()会暂停执行，并返回传入的值
    edited_text = interrupt(
        {
            "text_to_review": state["text"],
            "prompt": "请审查并编辑这段文本"
        }
    )
    
    print(f"✅ 收到编辑后的文本: {edited_text}")
    
    # 返回编辑后的文本
    return {"text": edited_text}


# 构建简单的图
simple_builder = StateGraph(SimpleState)
simple_builder.add_node("human_review", human_review_node)
simple_builder.add_edge(START, "human_review")
simple_builder.add_edge("human_review", END)

# 关键：必须添加checkpointer！
memory = MemorySaver()
simple_graph = simple_builder.compile(checkpointer=memory)

print("✅ 简单审查图构建完成")

In [ ]:
# 可视化
try:
    display(Image(simple_graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"无法显示图像: {e}")

In [ ]:
# 第一步：运行到中断点
config = {"configurable": {"thread_id": "test-1"}}

# result是interrupt返回的
result = simple_graph.invoke(
    {"text": "这是原始文本"},
    config
)

print(f"\n返回结果: {result}")

In [ ]:
# 检查图的状态
state = simple_graph.get_state(config)
print(f"当前节点: {state.next}")
print(f"中断数量: {len(state.tasks)}")
if state.tasks:
    print(f"中断详情: {state.tasks[0].interrupts}")

In [ ]:
# 第二步：使用Command恢复执行

# 使用Command提供人工编辑后的文本，resume的值会传给human_review_node中的edited_text
result = simple_graph.invoke(
    Command(resume="这是编辑后的文本，增加了新内容"),
    config
)

print(f"\n最终结果: {result}")
print(f"\n最终文本: {result['text']}")

## 3. 方式一：硬编码的人工审查节点

在这个模式中，我们在图中明确定义一个人工审查节点，并使用`Command`动态路由到不同节点。

In [ ]:
# 定义一个需要审查的工具
@tool
def delete_file(filename: str) -> str:
    """删除文件（模拟）
    
    Args:
        filename: 要删除的文件名
    """
    return f"文件 {filename} 已删除"


@tool  
def send_email(to: str, subject: str) -> str:
    """发送邮件（模拟）
    
    Args:
        to: 收件人
        subject: 邮件主题
    """
    return f"已发送邮件给 {to}，主题: {subject}"


sensitive_tools = [delete_file, send_email]
llm_with_sensitive_tools = llm.bind_tools(sensitive_tools)

print("✅ 定义了敏感操作工具")

In [ ]:
def agent_node(state: MessagesState):
    """Agent节点：调用LLM"""
    messages = [
        SystemMessage(content="你是一个助手，可以帮助用户管理文件和发送邮件。")
    ] + state["messages"]
    
    response = llm_with_sensitive_tools.invoke(messages)
    return {"messages": [response]}


def human_review_node_with_command(state: MessagesState):
    """人工审查节点：检查是否批准工具调用"""
    last_message = state["messages"][-1]
    
    if not hasattr(last_message, "tool_calls") or not last_message.tool_calls:
        return Command(goto=END)
    
    tool_call = last_message.tool_calls[0]
    
    # 中断并请求批准
    approval = interrupt(
        {
            "tool_name": tool_call["name"],
            "tool_args": tool_call["args"],
            "prompt": f"是否批准执行 {tool_call['name']}？请回复 'approve' 或 'reject'"
        }
    )
    
    if approval.lower() == "approve":
        # 批准：前往工具节点
        return Command(
            goto="tools"
        )
    else:
        # 拒绝：返回agent重新规划
        return Command(
            goto="agent"
        )


def tools_node(state: MessagesState):
    """工具节点：执行工具"""
    tools_by_name = {t.name: t for t in sensitive_tools}
    last_message = state["messages"][-1]
    
    results = []
    for tool_call in last_message.tool_calls:
        tool = tools_by_name[tool_call["name"]]
        result = tool.invoke(tool_call["args"])
        results.append(ToolMessage(
            content=str(result),
            tool_call_id=tool_call["id"]
        ))
    
    return {"messages": results}


print("✅ 节点函数定义完成")

In [ ]:
# 构建图
review_builder = StateGraph(MessagesState)

# 添加节点
review_builder.add_node("agent", agent_node)
review_builder.add_node("human_review", human_review_node_with_command)
review_builder.add_node("tools", tools_node)

# 添加边
review_builder.add_edge(START, "agent")
review_builder.add_edge("agent", "human_review")  # agent后总是需要审查
review_builder.add_edge("tools", "agent")  # 工具执行后返回agent

# 编译（必须有checkpointer）
review_memory = MemorySaver()
review_graph = review_builder.compile(checkpointer=review_memory)

print("✅ 审查图构建完成")

In [ ]:
# 可视化
try:
    display(Image(review_graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"无法显示: {e}")

In [ ]:
# 测试：删除文件场景
print("="*50)
print("场景1：请求删除文件")
print("="*50)

config = {"configurable": {"thread_id": "review-1"}}

result = review_graph.invoke(
    {"messages": [HumanMessage(content="请删除 old_data.txt 文件")]},
    config
)

# 检查中断
if "__interrupt__" in result:
    print("\n⏸️  执行已暂停，等待审批")
    print(f"中断信息: {result['__interrupt__']}")
    
    # 查看状态
    state = review_graph.get_state(config)
    print(f"\n当前停在: {state.next}")
    if state.tasks:
        print(f"等待决策: {state.tasks[0].interrupts}")

In [ ]:
# 批准操作
result = review_graph.invoke(
    Command(resume="approve"),
    config
)

print("\n对话历史:")
for msg in result["messages"]:
    msg.pretty_print()

In [ ]:
# 测试：拒绝场景
print("="*50)
print("场景2：拒绝发送邮件")
print("="*50)

config2 = {"configurable": {"thread_id": "review-2"}}

# 第一步：请求发送邮件
result = review_graph.invoke(
    {"messages": [HumanMessage(content="发送邮件给 boss@company.com，主题是'辞职信'")]},
    config2
)

if "__interrupt__" in result:
    print("⏸️  等待批准...")
    
    # 拒绝
    result = review_graph.invoke(
        Command(resume="reject"),
        config2
    )
    
    print("\n❌ 操作已拒绝")
    print("最后一条消息:")
    result["messages"][-1].pretty_print()

## 4. 方式二：LLM驱动的人工协同

在这个模式中，我们创建一个特殊的工具 `ask_human`，让LLM自己决定何时需要人工帮助。

In [ ]:
@tool
def search_web(query: str) -> str:
    """搜索网络（模拟）
    
    Args:
        query: 搜索查询
    """
    # 模拟搜索结果
    print("使用search_web工具")
    return f"关于'{query}'的搜索结果：这是一些模拟的搜索结果..."


@tool
def ask_human(question: str) -> str:
    """向用户请求帮助或澄清
    
    当你不确定用户意图、需要额外信息、或需要用户确认时使用此工具。
    
    Args:
        question: 要问用户的问题
    """
    # 这个工具内部会触发interrupt
    human_response = interrupt(
        {
            "question": question,
            "prompt": "请回答以下问题"
        }
    )
    return human_response


smart_tools = [search_web, ask_human]
llm_with_smart_tools = llm.bind_tools(smart_tools)

print("✅ 定义了智能工具")

In [ ]:
from langgraph.prebuilt import create_react_agent

# 使用预构建的agent，简化代码
smart_agent = create_react_agent(
    llm,
    tools=smart_tools,
    prompt="""你是一个智能助手。
    
当你遇到以下情况时，应该使用ask_human工具：
- 用户的需求不明确，需要澄清
- 你需要用户提供额外信息
- 执行重要操作前需要确认
- 你不确定应该怎么做


主动询问用户，不要猜测！
""",
    checkpointer=MemorySaver()  # 必须有checkpointer
)

print("✅ 智能Agent创建完成")

In [ ]:
# 可视化
try:
    display(Image(smart_agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"无法显示: {e}")

In [ ]:
# 测试：模糊请求
print("="*50)
print("测试：模糊的请求")
print("="*50)

config = {"configurable": {"thread_id": "smart-1"}}

result = smart_agent.invoke(
    {"messages": [HumanMessage(content="我需要一些关于天气的信息")]},
    config
)

# 检查是否触发了ask_human
if "__interrupt__" in result:
    print("\n🤔 Agent请求澄清！")
    interrupts = result["__interrupt__"]
    print(f"问题: {interrupts}")
    
    response = input(f"{interrupts[0].value['prompt']}:{interrupts[0].value['question']}")
    # response = "我想了解最新的天气预测技术，请用网络搜索工具检索最新信息"
    
    # 提供更具体的信息
    result = smart_agent.invoke(
        Command(resume=response), # resume的值会传给ask_human中的human_response
        config
    )
    
    print("\n最终回答:")
    result["messages"][-1].pretty_print()
else:
    print("Agent直接回答了")
    result["messages"][-1].pretty_print()

## 5. 状态持久化：跨会话恢复

### 为什么需要持久化？

在生产环境中：
- 服务器可能重启
- 用户可能稍后再回来
- 多个服务器之间需要共享状态

### MemorySaver vs 生产环境

- **MemorySaver**: 内存存储，仅用于开发/测试
- **生产环境**: 使用Redis、PostgreSQL等持久化存储

In [ ]:
# 演示状态持久化
print("="*50)
print("演示：状态持久化")
print("="*50)

# 创建一个新的agent
persistent_memory = MemorySaver()
persistent_agent = create_react_agent(
    llm,
    tools=[search_web],
    checkpointer=persistent_memory
)

# 第一次对话
thread_id = "user-123"
config = {"configurable": {"thread_id": thread_id}}

print("第一次对话：")
result1 = persistent_agent.invoke(
    {"messages": [HumanMessage(content="搜索LangGraph教程")]},
    config
)
print(f"消息数量: {len(result1['messages'])}")

# 模拟：过了一段时间...
print("\n第二次对话（使用相同thread_id）：")
result2 = persistent_agent.invoke(
    {"messages": [HumanMessage(content="刚才搜索的是什么？")]},
    config
)
print(f"消息数量: {len(result2['messages'])}")
print("\nAgent的回答:")
result2["messages"][-1].pretty_print()

print("\n✅ Agent记住了之前的对话！")

In [ ]:
# 查看完整的状态历史
print("状态历史记录：")
print("="*50)

# 获取所有检查点
history = list(persistent_agent.get_state_history(config))
print(f"共有 {len(history)} 个检查点\n")

for i, checkpoint in enumerate(history[:5]):  # 只显示前5个
    print(f"检查点 {i}:")
    print(f"  - 节点: {checkpoint.metadata.get('step', 'N/A')}")
    print(f"  - 消息数: {len(checkpoint.values.get('messages', []))}")
    print()

## 6. 时间旅行：回到过去的状态

LangGraph的checkpointer不仅能保存状态，还能回到历史中的任意检查点！

In [ ]:
# 回到某个历史状态
print("="*50)
print("时间旅行：回到过去")
print("="*50)

# 获取历史
history = list(persistent_agent.get_state_history(config))

if len(history) >= 3:
    # 选择第3个检查点
    old_state = history[2]
    print(f"回到检查点: {old_state.config['configurable']}")
    print(f"该状态的消息数: {len(old_state.values['messages'])}")
    
    # 从这个状态继续执行
    result = persistent_agent.invoke(
        {"messages": [HumanMessage(content="你好，我是新的问题")]},
        old_state.config
    )
    
    print("\n从过去的状态继续执行成功！")
else:
    print("历史记录不足，请先运行更多对话")

## 7. 核心概念总结

### interrupt() 工作原理

1. **暂停执行**: `interrupt(value)` 抛出特殊异常，暂停图的执行
2. **保存状态**: checkpointer保存当前状态
3. **返回值**: `value`被包装在`__interrupt__`中返回
4. **等待恢复**: 执行暂停，等待 `Command(resume=...)`
5. **继续执行**: resume的值作为interrupt()的返回值

### Command 的作用

```python
Command(
    goto="next_node",    # 跳转到哪个节点
    update={...},         # 更新状态
    resume=value          # 恢复中断
)
```

### Checkpointer 必要性

| 功能 | 需要checkpointer |
|------|------------------|
| interrupt() | ✅ 必须 |
| 多轮对话记忆 | ✅ 必须 |
| 时间旅行 | ✅ 必须 |
| 基础执行 | ❌ 不需要 |

### 两种人工协同模式对比

| 特性 | 硬编码审查 | LLM驱动 |
|------|------------|----------|
| 控制力 | 高 | 中 |
| 灵活性 | 低 | 高 |
| 实现复杂度 | 高 | 低 |
| 适用场景 | 固定流程 | 动态需求 |

## 8. 练习题

### 练习1: 多步审批
修改审查流程，要求重要操作需要两个人批准。

### 练习2: 超时机制
实现一个超时机制：如果用户30秒内没有响应，自动拒绝操作。

### 练习3: 审批日志
在state中添加一个审批历史列表，记录所有的审批决策。

### 思考题
1. 为什么interrupt()必须配合checkpointer使用？
2. 如果在生产环境中，应该选择什么样的checkpointer?
3. Command的goto和普通edge有什么区别？

## 9. 生产环境建议

### Checkpointer选择

```python
# 开发/测试
from langgraph.checkpoint.memory import MemorySaver
checkpointer = MemorySaver()

# 生产环境 - Redis
# from langgraph.checkpoint.redis import RedisSaver
# checkpointer = RedisSaver.from_conn_string("redis://localhost:6379")

# 生产环境 - PostgreSQL
# from langgraph.checkpoint.postgres import PostgresSaver
# checkpointer = PostgresSaver.from_conn_string("postgresql://...")
```

### 安全注意事项

1. **验证用户身份**: 确保只有授权用户才能批准操作
2. **记录审批日志**: 保存完整的审批历史
3. **超时处理**: 设置合理的超时时间
4. **敏感信息保护**: 不要在interrupt中暴露敏感数据

## 10. 下一步

在下一章（第3章），我们将学习：
- 如何创建SubAgent
- 上下文隔离
- 并发执行
- 结果合成